In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, lit, isnull
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

invoice = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "invoice") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

invoice.show()

24/10/06 17:44:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+---------+
|serial_no|
+---------+
|   330115|
|   330120|
|   330121|
|   330122|
|   330125|
+---------+



In [3]:
min_serial = invoice.agg({"serial_no": "min"}).collect()[0][0]
max_serial = invoice.agg({"serial_no": "max"}).collect()[0][0]

# Generate a DataFrame with the sequence of numbers from min to max
full_sequence_df = spark.range(min_serial, max_serial + 1).toDF("all_invoice")
full_sequence_df\
    .join(invoice,on=full_sequence_df.all_invoice == invoice.serial_no,how="left")\
    .filter(col("serial_no").isNull())\
    .select("all_invoice")\
    .show()


+-----------+
|all_invoice|
+-----------+
|     330116|
|     330117|
|     330118|
|     330119|
|     330123|
|     330124|
+-----------+

